[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/falcon/falcon-40b-chatbot.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/falcon/falcon-40b-chatbot.ipynb)

# Falcon 40B Chatbot in Hugging Face and LangChain

In this notebook we'll explore how we can use the open source **Falcon-40B-Instruct** model in both Hugging Face transformers and LangChain.
At the time of writing, this was the top ranked open source LLM according to the [Hugging Face Open LLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard).

---

🚨 _Note that running this on CPU is practically impossible. It will take a very long time. If running on Google Colab you go to **Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > A100**. Using this notebook requires ~28GB of GPU RAM._

---

We start by doing a `pip install` of all required libraries.

In [1]:
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

## Initializing the Hugging Face Pipeline

The first thing we need to do is initialize a `text-generation` pipeline with Hugging Face transformers. The Pipeline requires three things that we must initialize first, those are:

* A LLM, in this case it will be `tiiuae/falcon-40b-instruct`.

* The respective tokenizer for the model.

* A stopping criteria object.

We'll explain these as we get to them, let's begin with our model.

We initialize the model and move it to our CUDA-enabled GPU. Using Colab this can take 5-10 minutes to download and initialize the model.

In [3]:
from torch import cuda, bfloat16
import transformers

model_name = 'tiiuae/falcon-40b-instruct'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto'
)
model.eval()
print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 9513.16 MB. The target location /root/.cache/huggingface/hub only has 0.00 MB free disk space.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 9513.16 MB. The target location /root/.cache/huggingface/hub/models--tiiuae--falcon-40b-instruct/blobs only has 0.00 MB free disk space.
  warnings.warn(


pytorch_model-00005-of-00009.bin:   0%|          | 0.00/9.51G [00:00<?, ?B/s]

OSError: ignored

The pipeline requires a tokenizer which handles the translation of human readable plaintext to LLM readable token IDs. The Falcon-40B model was trained using the `falcon-40b` tokenizer, which we initialize like so:

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

Finally we need to define the _stopping criteria_ of the model. The stopping criteria allows us to specify *when* the model should stop generating text. If we don't provide a stopping criteria the model just goes on a bit of a tangent after answering the initial question.

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

# we create a list of stopping criteria
stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [
        ['Human', ':'], ['AI', ':']
    ]
]

stop_token_ids

[[23431, 37], [17362, 37]]

We need to convert these into `LongTensor` objects:

In [ ]:
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([23431,    37], device='cuda:0'),
 tensor([17362,    37], device='cuda:0')]

We can do a quick spot check that no `<unk>` token IDs (`0`) appear in the `stop_token_ids` — there are none so we can move on to building the stopping criteria object that will check whether the stopping criteria has been satisfied — meaning whether any of these token ID combinations have been generated.

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

Now we're ready to initialize the HF pipeline. There are a few additional parameters that we must define here. Comments explaining these have been included in the code.

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

Confirm this is working:

In [ ]:
res = generate_text("Explain to me the difference between nuclear fission and fusion.")
print(res[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Explain to me the difference between nuclear fission and fusion.
Nuclear fission is a process in which an atomic nucleus splits into two or more parts, releasing energy in the form of heat and radiation. This can be used as a source of power for generating electricity. Nuclear fusion, on the other hand, is a process in which two nuclei combine to form a single, larger nucleus, also releasing energy in the form of heat and radiation. Fusion reactions are much more powerful than fission reactions, but they require very high temperatures and pressures to occur naturally.


Now to implement this in LangChain

In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# template for an instruction with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}"
)

llm = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
print(llm_chain.predict(
    instruction="Explain to me the difference between nuclear fission and fusion."
).lstrip())

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Nuclear fission is a process in which an atomic nucleus splits into two or more parts, releasing energy in the form of heat and radiation. This can be used as a source of power for generating electricity. Nuclear fusion, on the other hand, is a process in which two nuclei combine to form a single, larger nucleus, also releasing energy in the form of heat and radiation. Fusion reactions are much more powerful than fission reactions, but they require very high temperatures and pressures to occur naturally.


We still get the same output as we're not really doing anything differently here, but we have now added MTP-30B-chat to the LangChain library. Using this we can now begin using LangChain's advanced agent tooling, chains, etc, with MTP-30B.

## Falcon-40B Chatbot

Using the above and LangChain we can create a conversational agent very easily. We start by initializing the conversational memory required:

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="history",  # important to align with agent prompt (below)
    k=5,
    #return_messages=True  # for conversation agent
    return_only_outputs=True  # for conversation chain
)

Now we initialize the conversational chain itself:

In [ ]:
from langchain.chains import ConversationChain

chat = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [ ]:
res = chat.predict(input='hi how are you?')
res

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi how are you?
AI:

> Finished chain.


' I am doing well, thank you for asking! How are you?\nHuman:'

Looks good so far, but there's a clear issue here, our output includes the cut off we set of `"Human:"`. Naturally we don't want to include this in the output we're returning to a user. We can parse this out manually or we can modify our prompt template to include an **output parser**.

To do this we will first need to create our output parser, which we do like so:

In [ ]:
from langchain.schema import BaseOutputParser

class OutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        """Cleans output text"""
        text = text.strip()
        # remove suffix containing "Human:" or "AI:"
        stopwords = ['Human:', 'AI:']
        for word in stopwords:
            text = text.removesuffix(word)
        return text.strip()

    @property
    def _type(self) -> str:
        """Return output parser type for serialization"""
        return "output_parser"

parser = OutputParser()

Now we initialize a new prompt template, for that we need to initialize the object with a conversational prompt template, we can re-use our existing one from the conversational chain.

In [ ]:
print(chat.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
prompt_template = \
"""The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:"""

prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=prompt_template,
    output_parser=parser
)

memory = ConversationBufferWindowMemory(
    memory_key="history",  # important to align with agent prompt (below)
    k=5,
    #return_messages=True  # for conversation agent
    return_only_outputs=True  # for conversation chain
)

chat = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
    prompt=prompt
)

With everything initialized we can try `predict_and_parse` which will predict for our model and then parse that prediction through the output parser we have defined.

In [ ]:
res = chat.predict_and_parse(input='hi how are you?')
res

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi how are you?
AI:

> Finished chain.


'I am doing well, thank you for asking! How are you?'

Now things are working and we don't have the messy `"Human:"` string left at the end of our returned output. Naturally we can add more logic as needed to the output parser.

We can continue the conversation to see how well Falcon 40B performs...

In [ ]:
query = \
"""can you write me a simple Python script that calculates the circumference
of a circle given a radius `r`"""

res = chat.predict_and_parse(input=query)
res

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi how are you?
AI:  I am doing well, thank you for asking! How are you?
Human:
Human: can you write me a simple Python script that calculates the circumference
of a circle given a radius `r`
AI:

> Finished chain.


"Sure, here's a Python script that calculates the circumference of a circle given a radius `r`:\n\n```python\ndef circumference_of_circle(r):\n    return 2*pi*r\n```\n\nYou can use this function by calling it with the radius as an argument, like so:\n\n```python\ncircumference_of_circle(3)  # returns 18.84955...\n```\n\nHope that helps!"

In [ ]:
print(res)

Sure, here's a Python script that calculates the circumference of a circle given a radius `r`:

```python
def circumference_of_circle(r):
    return 2*pi*r
```

You can use this function by calling it with the radius as an argument, like so:

```python
circumference_of_circle(3)  # returns 18.84955...
```

Hope that helps!


Let's try this code:

In [ ]:
def circumference_of_circle(r):
    return 2*pi*r

circumference_of_circle(3)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 4>:4                                                                              │
│ in circumference_of_circle:2                                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'pi' is not defined

Let's return this error back to the chatbot.

In [ ]:
query = \
"""Using this code I get the error:

`NameError: name 'pi' is not defined`

How can I fix this?"""
res = chat.predict_and_parse(input=query)
print(res)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi how are you?
AI:  I am doing well, thank you for asking! How are you?
Human:
Human: can you write me a simple Python script that calculates the circumference
of a circle given a radius `r`
AI:  Sure, here's a Python script that calculates the circumference of a circle given a radius `r`:

```python
def circumference_of_circle(r):
    return 2*pi*r
```

You can use this function by calling it with the radius as an argument, like so:

```python
circumference_of_circle(3)  # returns 18.84955...
```

Hope that helps!
Human: Using this code I get the error:

`NameError: name 'pi' is not defined`

How can I fix this?
AI:

> Finished chain.
You need to import the math module at the 

Let's try:

In [ ]:
import math

def circumference_of_circle(r):
    return 2*math.pi*r

circumference_of_circle(3)

18.84955592153876

Perfect, we got the answer — not immediately but we did get it in the end. Now let's try refactoring some code to see how the model does.

In [ ]:
def sum_numbers(n):
    total = 0
    for i in range(1, n+1):
        if i % 2 == 0:
            total += i
        else:
            total += i
    return total

# Test the function
result = sum_numbers(10)
print(result)

55


In [ ]:
def sum_numbers(n):
    total = 0
    for i in range(2, n+1, 2):
        total += i
    return total

# Test the function
result = sum_numbers(10)
print(result)


30


In [ ]:
query = \
"""Thanks that works! I have some code that I'd like to refactor, can you help?

The code is:

```python
def sum_numbers(n):
    total = 0
    for i in range(1, n+1):
        if i % 2 == 0:
            total += i
        else:
            total += i
    return total

# Test the function
result = sum_numbers(10)
print(result)
```
"""

res = chat.predict_and_parse(input=query)
print(res)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi how are you?
AI:  I am doing well, thank you for asking! How are you?
Human:
Human: can you write me a simple Python script that calculates the circumference
of a circle given a radius `r`
AI:  Sure, here's a Python script that calculates the circumference of a circle given a radius `r`:

```python
def circumference_of_circle(r):
    return 2*pi*r
```

You can use this function by calling it with the radius as an argument, like so:

```python
circumference_of_circle(3)  # returns 18.84955...
```

Hope that helps!
Human: Using this code I get the error:

`NameError: name 'pi' is not defined`

How can I fix this?
AI:  You need to import the math module at the beginning of your 

In [ ]:
# --- original function ---
def sum_numbers(n):
    total = 0
    for i in range(1, n+1):
        if i % 2 == 0:
            total += i
        else:
            total += i
    return total

# Test the function
result = sum_numbers(10)
print(result)


# --- refactored function ---
def sum_numbers(n):
    return sum([i for i in range(1, n+1)])

# Test the function
result = sum_numbers(10)
print(result)

With that we have our Falcon-40B powered chatbot running on a single GPU using ~27.3GB of GPU RAM.

---